In [84]:
import os

In [86]:
import datasets

In [87]:
local_dir = '/cwStorage/nodecw_group/jijh/hest_1k'

In [89]:
help(datasets.load_dataset)

Help on function load_dataset in module datasets.load:

load_dataset(path: str, name: Optional[str] = None, data_dir: Optional[str] = None, data_files: Union[str, Sequence[str], Mapping[str, Union[str, Sequence[str]]], NoneType] = None, split: Union[str, datasets.splits.Split, NoneType] = None, cache_dir: Optional[str] = None, features: Optional[datasets.features.features.Features] = None, download_config: Optional[datasets.download.download_config.DownloadConfig] = None, download_mode: Union[datasets.download.download_manager.DownloadMode, str, NoneType] = None, verification_mode: Union[datasets.utils.info_utils.VerificationMode, str, NoneType] = None, keep_in_memory: Optional[bool] = None, save_infos: bool = False, revision: Union[str, datasets.utils.version.Version, NoneType] = None, token: Union[bool, str, NoneType] = None, streaming: bool = False, num_proc: Optional[int] = None, storage_options: Optional[Dict] = None, trust_remote_code: bool = None, **config_kwargs) -> Union[datas

In [98]:
import datasets

# 数据集脚本的本地路径
dataset_script_path = '/cwStorage/nodecw_group/jijh/hest_1k/hest.py'

# 本地数据集路径
data_dir = '/cwStorage/nodecw_group/jijh/hest_1k'

# 加载本地数据集脚本
dataset = datasets.load_dataset(
    dataset_script_path,
    data_dir=data_dir,
    name="custom_config"  # 使用自定义配置名称
)

# 示例：查看数据集的训练集
print(dataset['train'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating examples: 100%|██████████| 44883/44883 [00:00<00:00, 139178.48it/s]

Dataset({
    features: ['path'],
    num_rows: 44883
})


In [100]:
dataset['train'][]

Dataset({
    features: ['path'],
    num_rows: 44883
})